# 3 - Vector Search and LLM prompting

This notebook shows how we can query our vector database using plain-text queries.

**Before starting, I want to emphasize that the patient data we are using is all synthetically generated. The main FHIR resources were created by [Synthea](https://synthetichealth.github.io/synthea/), while the clinical notes being used in this example are created by Copilot. 

As in previous steps, we will first create our iris cursor:


In [1]:
import iris
conn = iris.connect("localhost", 32782, "DEMO", "_SYSTEM", "ISCDEMO") # Server, Port , Namespace, Username, Password
cursor = conn.cursor()

The database search is performed using functions built into IRIS-SQL. The query, which is given in plain text, is then encoded into a vector using the same sentence transformer model as used for the information in the database. Here we are going to simply query the database about headaches:

Importing the model will take a short time, but encoding the query into embeddings should be quick. 

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2') 
query = "Has the patient reported any chest or respiratory complaints?"
query_vector = model.encode(query, normalize_embeddings=True, show_progress_bar = False).tolist()

There are two ways to measure the similarity between vectors, we can look at the vector cosine or the dot-product. The cosine is the angle between the two vectors, while the dot product is the distance between them. They have slight differences that I won't go into much detail here, but can be explored if you want to optimise your search. In reality, for small searches, the results will be similar whichever method you use. 

In [3]:
table_name = "VectorSearch.DocRefVectors"
search_sql = f"""
        SELECT TOP 3 ClinicalNotes 
        FROM {table_name}
        WHERE PatientID = ?
        ORDER BY VECTOR_COSINE(NotesVector, TO_VECTOR(?,double)) DESC
    """

cursor.execute(search_sql, [3, str(query_vector)])
results = cursor.fetchall()

for result in results:
    print("\n--------------------------------------Note Start---------------------------------")
    print(result[0])
    print("--------------------------------------Note End---------------------------------\n")


--------------------------------------Note Start---------------------------------
Date: 2022-06-16
Provider: Dr. Fritz267 Lebsack687
Location: PCP114062
Reason for Visit: Routine general examination
Clinical Summary:
Alisia attended her scheduled wellness check. She reported feeling generally well, with no acute complaints. Her medical history includes resolved streptococcal pharyngitis and viral sinusitis. She continues to manage her weight and blood pressure effectively.
Observations:

Height: 172.7 cm
Weight: 81.2 kg
BMI: 27.23 kg/m²
Pain Score: 2/10 (mild discomfort reported in lower back)

Assessment:

No new diagnoses
Stable vitals and weight
Mild musculoskeletal discomfort, likely posture-related

Plan:

Continue current lifestyle and dietary habits
Encourage regular stretching and core strengthening exercises
Schedule next routine exam in 12 months unless symptoms arise

Billing:

General examination
Submitted to Medicare
Total: $129.16
--------------------------------------No

This SQL query selects the top  result from a vector search of our table. The results are ordered by the vector cosine of the NotesVector column of the database and the vector of our query - we have to order in descending order to get the best match first. 

Note, we have an additional filter here to only include the results for a particular patient ID which will be provided upon execution. We could provide any number of additional filters, for example each clinical note starts with a date. In the set-up section we could extract this date and save it as a separate column within our data table. Then when querying the database we could add a query to only include a particular date range. 

Below I have put this into an (almost) standalone function. This function includes the global variables model and table_name defined above. 

In [ ]:
def vector_search(user_prompt,patient):
    search_vector =  model.encode(user_prompt, normalize_embeddings=True,show_progress_bar=False ).tolist() 
    
    search_sql = f"""
        SELECT TOP 3 ClinicalNotes 
        FROM {table_name}
        WHERE PatientID = ?
        ORDER BY VECTOR_COSINE(NotesVector, TO_VECTOR(?,double)) DESC
    """
    cursor.execute(search_sql,[patient, str(search_vector)])
    
    results = cursor.fetchall()
    return results

In [5]:
results = vector_search(query, 3)
print(results)

(('Date: 2022-06-16\nProvider: Dr. Fritz267 Lebsack687\nLocation: PCP114062\nReason for Visit: Routine general examination\nClinical Summary:\nAlisia attended her scheduled wellness check. She reported feeling generally well, with no acute complaints. Her medical history includes resolved streptococcal pharyngitis and viral sinusitis. She continues to manage her weight and blood pressure effectively.\nObservations:\n\nHeight: 172.7 cm\nWeight: 81.2 kg\nBMI: 27.23 kg/m²\nPain Score: 2/10 (mild discomfort reported in lower back)\n\nAssessment:\n\nNo new diagnoses\nStable vitals and weight\nMild musculoskeletal discomfort, likely posture-related\n\nPlan:\n\nContinue current lifestyle and dietary habits\nEncourage regular stretching and core strengthening exercises\nSchedule next routine exam in 12 months unless symptoms arise\n\nBilling:\n\nGeneral examination\nSubmitted to Medicare\nTotal: $129.16',), ('Date: 2021-06-20\nProvider: Dr. Yael855 Hartmann983\nLocation: MILFORD REGIONAL MEDIC

### Prompting a Local LLM 

There are a few options when it comes to prompting an LLM. Most LLMs available are accessible through APIs, for example you can use GPT-5 through [OpenAI's API](https://openai.com/api/). For sensitive data, like Patient FHIR data, there might be data protection rules or even laws which would prohibit sending sensitive data to an external API, so this might not be possible. In these cases it is possible to download a local version of models. Local versions can be downloaded with Ollama, or very simply with hugging faces. 

#### Using hugging faces transformer

This is the easiest way to get started with a local LLM. Transformers can be installed using pip and then simply used as shown below.

I'm using a very small model to make it lighter-weight for use on my laptop, but don't expect very good results using this method unless you are willing to download larger models. For better results using local models, feel free to skip to the Ollama section below, which requires slightly more set-up but is giving much better results. 

In [6]:
# pip install transformers

In [7]:
from transformers import pipeline
# Create our chatbot using the flan-t5-base model
generator = pipeline("text2text-generation", model="google/flan-t5-base", max_length=512, min_length=100)

Device set to use cpu


In [8]:
## Perform Vector Search again
query = "Has the patient reported having bad headaches?"
results = vector_search(query, 3)
print("Vector search complete") 

prompt = (
    f"""SYSTEM: You are a helpful and knowledgeable assistant designed to help a doctor interpret a patient's medical history 
    using retrieved information from a database. Please provide a detailed and medically relevant explanation and include relevant
    dates in your response and ensure your response is coherent \n\n\n"""
    f"CONTEXT:\n{results[:-1]}\n\n"
    f"USER QUESTION:\n{query}\n\n"
    f"ASSISTANT ANSWER:"
)
response = generator(prompt)
print(response[0]["generated_text"])

Token indices sequence length is longer than the specified maximum sequence length for this model (634 > 512). Running this sequence through the model will result in indexing errors


Vector search complete
The patient reported persistent nasal congestion, facial pressure, and postnasal drip lasting over a week. She denied fever, purulent discharge, or severe headache. Physical exam showed mild sinus tenderness and clear nasal secretions. No signs of bacterial infection were present. Examination showed mild sinus tenderness and clear nasal secretions. No signs of bacterial superinfection were present.nDiagnosis:nnViral sinusitis (active)nTreatment Plan:nnSupportive care: saline nasal irrigation, steam inhalation, and OTC decongestantsnAvoid antibiotics due to viral etiologynAdvised to monitor for worsening symptoms or persistence beyond 10 daysnProvided education on differentiating viral vs. bacterial sinusitisnnAdvised to monitor for worsening symptoms or persistence beyond 10 daysnEducated on signs of bacterial superinfectionnnBilling:nnEncounter for symptom and viral sinusitisnSubmitted to MedicarenTotal: $129.16',)


So the vector search gave an answer which was relevant to the prompt, but ignored some parts of it (notably, I added in the system prompt that it should include dates in its response). This is a limitation of using a small model. 

### Prompting a local model with Ollama

Here we are going to use [Ollama](https://ollama.com/) and in particular the [gemma3:1b](https://ollama.com/library/gemma3) model. To start using Ollama: 

1. To start using Ollama, download Ollama from the [Ollama Website](https://ollama.com/). 
2. After downloading Ollama, download the required model 
    - we are going to use `gemma3:1b`, one of the best small models. If you have very limited computing power or harddrive capacity, feel free to change model to an even smaller one like `gemma3:270m`, which is <300MB in size. For more notes
    - To download, select it from the Ollama GUI or use the terminal command `ollama pull gemma3:1b`.
3. Download the companion python library: `pip install ollama`


In [ ]:
pip install ollama

In [9]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='gemma3:1b', messages=[{'role': 'system',
            'content': (
                "You are a helpful and knowledgeable assistant designed to help a doctor interpret a patient's medical history using retrieved information from a database."
                "Please provide a detailed and medically relevant explanation, include relevant dates, and ensure your response is coherent."
            )},
        {
            'role': 'user',
            'content': f"CONTEXT:\n{results}\n\nUSER QUESTION:\n{query}"
        }])
print("\n==================================Response Start===============================\n")
print(response['message']['content'])
print("\n==================================Response End===============================\n")


==================================Response Start===============================

Okay, let's analyze the provided medical records to address your question.

Based on the information available, the patient, Alisia, *does not* report having any history of headaches.

Here’s a breakdown of the relevant information:

*   **2021-06-20 (Viral Sinusitis - Active):** The record mentions postnasal drip and clear nasal secretions, which is consistent with a viral infection. There’s no mention of headaches.
*   **2021-06-20 (Encounter for Symptom and Viral Sinusitis):** The record emphasizes supportive care measures (saline irrigation, steam inhalation, OTC decongestants) – these are treatments for symptoms, not headaches.
*   **2022-06-16 (Routine General Examination):** The record focuses on general well-being, weight, and musculoskeletal discomfort. It does not include any information about headaches.

**Conclusion:**  The provided records do not indicate that Alisia has reported experiencing

Thats already a better response than the basic hugging faces one. This could still get much better, as `gemma3:1b` is a very small model (<1Gb in size, 1 billion parameters). This is one of the smallest usable models available. Larger models will generally give better results, whilst also allowing longer prompts with more search results. However, large models and long prompts also comes with the trade-off of requiring increased processing power.

Theres  several versions of `gemma3` with increasing numbers of parameters that could give improved results. You can see a list of all available models on the [ollama website](https://ollama.com/library). A shortened list of very small models is availble below: 

|Model Name|# Parameters (Billions)|Size(GB)|
|-|-|-|
|gemma3:1b|1|0.8|
|gemma3:4b|4|3.3|
|gemma3:12b|12|1|
|qwen3:0.6b|0.6|0.5|
|qwen3:4b|4|2.5|
|mistral|7|4.1|
|phi4-mini:3.8b|3.8|2.5|

For any of these models, you can download them from the Ollama GUI or from the command line with `ollama pull <model-name>`. 
Different models are good at different things, for example some are specifically good at writing code, others have multi-model capability and can 'look at' images, or 'hear' audio. One thing to consider is that we need a model with a large context window, as this is the amount of information that can be included in the prompt. Sending the data retrieved by the vector search means that our prompts will end up being large in size.

### Adding Memory

When we chat with a chatbot, it's ideal for the model to remember the conversation that has come before. Memory can be implemented by passing the previous messages back to the chatbot with our next queries. Instead though, we going to use [LangChain](https://python.langchain.com/docs/introduction/) to automate this. 

LangChain cane automatically create summaries of the conversations rather than passing the whole message into the chatbot, reducing the length of the prompt to the model. This reduction can speed up the response, and reduce the amount of tokens used (and with API chatbot access, you pay per token). It is common to use a small model as a summariser, with a larger, better model for creating the actual response.

The first step is to install langchain and langchain community with pip. Uncomment and run the following:

In [10]:
#pip install langchain langchain-ollama 

Here we do the following: 
    - Import dependencies
    - Initialise our model with Ollama
    - Create an agent, this agent does not have any "tools". Discussing tools and agentic AI is beyond the scope of this tutorial, so for now you can ignore this. 
    - Initialise agent summarisation

In [11]:
# Import dependencies
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_ollama import ChatOllama

# Initialize model
llm = ChatOllama(model="gemma3:1b") 

# Initialise short-term memory
checkpointer = InMemorySaver()

# Create model
agent = create_agent(
    model=llm, # Set model as our LLM 
    middleware=[
        # create summarization proceedure - this creates summaries of our conversation to keep memory brief.
        SummarizationMiddleware(
            model=llm,
            max_tokens_before_summary=4000,  # Trigger summarization at 4000 tokens
            messages_to_keep=20,  # Keep last 20 messages after summary
        )
    ],
    # Creates the agent's memory with pre-initialized model
    checkpointer=checkpointer,
)


config = {"configurable": {"thread_id": "1"}}

agent.invoke({"messages": [{"role":"user", "content":"Hello, my name is Gabriel"}]}, config)
final_response = agent.invoke({"messages": [{"role":"user", "content":"what did I just tell you?"}]}, config)

for message in final_response["messages"]:
    message.pretty_print()


================================ Human Message =================================

Hello, my name is Gabriel
================================== Ai Message ==================================

Hello Gabriel! It's nice to meet you. How are you today? 😊 Is there anything you'd like to chat about?
================================ Human Message =================================

what did I just tell you?
================================== Ai Message ==================================

You just told me your name is Gabriel! 😄 

That’s a pretty cool name – it’s a very common name. 😊 

Is there anything you'd like to tell me about yourself, or would you like to talk about something?


The model has memory!

### Putting this together

Now lets put the whole chat architecture into a single class to make it easy to run. 

In [ ]:
import iris
class RAGChatbot:
    def __init__(self):
        self.message_count = 0
        conn = iris.connect("localhost", 32782, "DEMO", "_SYSTEM", "ISCDEMO") # Server, Port , Namespace, Username, Password
        self.cursor = conn.cursor()
        self.agent = self.create_agent()
        self.embedding_model = self.get_embedding_model()
        

    
    def get_embedding_model(self):
        return  SentenceTransformer('all-MiniLM-L6-v2') 
        
    def create_agent(self):



        # Initialize model
        llm = ChatOllama(model="gemma3:1b") 
        
        # Initialise short-term memory
        checkpointer = InMemorySaver()
        
        # Create model
        agent = create_agent(
            model=llm, # Set model as our LLM 
            middleware=[
                # create summarization proceedure - this creates summaries of our conversation to keep memory brief.
                SummarizationMiddleware(
                    model=llm,
                    max_tokens_before_summary=4000,  # Trigger summarization at 4000 tokens
                    messages_to_keep=20,  # Keep last 20 messages after summary
                )
            ],
            # Creates the agent's memory with pre-initialized model
            checkpointer=checkpointer,
        )
        self.config = {"configurable": {"thread_id": "1"}}
        return agent
        
    def vector_search(self, user_prompt,patient):
        search_vector =  self.embedding_model.encode(user_prompt, normalize_embeddings=True, show_progress_bar=False).tolist() 
        
        search_sql = f"""
            SELECT TOP 3 ClinicalNotes 
            FROM VectorSearch.DocRefVectors
            WHERE PatientID = ?
            ORDER BY VECTOR_COSINE(NotesVector, TO_VECTOR(?,double)) DESC
        """
        self.cursor.execute(search_sql,[patient, str(search_vector)])
        
        results = self.cursor.fetchall()
        return results

    def run(self):
        if self.message_count==0:

            
            query = input("\n\nHi, I'm a chatbot used for searching a patient's medical history. How can I help you today? \n\n - User: ")
        else:
            query = input("\n - User:")
        search = True
        if self.message_count != 0:
            search_ans = input("Search the database? [Y/N - default N]")
            if search_ans.lower() != "y":
                search = False

        if search:
            try:
                patient_id = int(input("What is the patient ID?"))
            except:
                print("The patient ID should be an integer")
                return

            results = self.vector_search(query, patient_id)
            if results == []:
                print("No results found, check patient ID")
                return

            prompt = f"CONTEXT:\n{results}\n\nUSER QUESTION:\n{query}"
        else:
            prompt = f"USER QUESTION:\n{query}"

        ##print(prompt)
        system_prompt = "You are a helpful and knowledgeable assistant designed to help a doctor interpret a patient's medical history using retrieved information from a database.\
        Please provide a detailed and medically relevant explanation, \
        include the dates of the information you are given."
        response = self.agent.invoke({"messages" : [("system", system_prompt), ("user", query), ("system", str(results))]}, self.config)
        response["messages"][-1].pretty_print()
        self.message_count += 1



In [24]:
# Usage
bot = RAGChatbot()
bot.run()



Hi, I'm a chatbot used for searching a patient's medical history. How can I help you today? 

 - User:   Does the patient have any history of respiratory issues? 
What is the patient ID? 3


================================== Ai Message ==================================

Okay, let’s analyze this patient’s medical history and determine if there’s a history of respiratory issues.

**Analysis of the Provided Information:**

Based on the retrieved data, Alisia’s history of respiratory issues appears to be **limited but significant**, primarily focused on viral sinusitis. Here's a breakdown:

*   **2021-06-20:**  Her initial diagnosis of viral sinusitis, *active*, indicates a recent respiratory issue.  The "Postnasal drip" and facial pressure symptoms are consistent with this diagnosis.
*   **2021-06-20:** The treatment plan specifically mentions "viral sinusitis," reinforcing this connection.
*   **2022-06-16:** The routine examination doesn't mention any respiratory symptoms or history. 

**Conclusion:**

The patient's history *primarily* centers around viral sinusitis.  The key takeaway is that she's been diagnosed with this condition, and the treatment plan reflects a mana

In [ ]:
bot.run()


 - User: How about any reports of fatigue? 
Search the database? [Y/N - default N] y
What is the patient ID? 3


================================== Ai Message ==================================

Okay, let’s break down Alisia’s fatigue status based on the provided information.

**Analysis of Fatigue:**

The patient’s fatigue is a significant point to consider, and it’s a layered assessment requiring more context. Here’s a detailed breakdown:

*   **Reported Fatigue:** Alisia *does* report fatigue, specifically mentioning it in her clinical summary. The severity is described as “mild,” which is important – it’s not a debilitating level.
*   **Timing:** The fatigue is *likely* age-related, a crucial piece of information.  It’s consistent with typical fatigue experienced as individuals age.
*   **Associated Symptoms:** Her history of “mild joint stiffness in the mornings” adds a layer of potential concern. While joint stiffness itself isn't directly linked to fatigue, it *could* be a symptom of underlying musculoskeletal issues (like postural strain or inflammation), which *could* contribute to fatig

# Conclusions

In this example, we are using synthetically generated clinical notes created by by a large language model. We have used a very small dataset, with only three patients with only 10 clinical notes each. However, it does show the potential of the process. You can hopefully see how this could be scaled up into a useful resource. 

#### Where to continue from here :

If this were a real project, rather than a quick demo, a number of improvements could be made:

- #### Vector Search
    - **Use a Medical Specific Embedding Model.** 
    - **Create a Score requirement**
        - The relevance of a search result to the prompt can be seen by the vector-cosine with a number between 1 and -1. You can use this to create a minimum requirement to classify a result as a hit.
    - **Filter by Date**
        - We are querying a patients entire medical history, which means we could get results from 50 years ago which obscure results from 6 months ago due to a slightly better semantic match. In reality we would probably be more interested in recent results, even if they weren't quite as good a match as the older ones.
        - We could approach this in different ways - we could just add a date restriction (e.g. search only in last 5 years) or we could take the top N results when ordered by score and then order them by date.

- #### LLM prompting
    - Use a better LLM
    - Refine the system prompt

- #### General set-up or design:

    - **Link source information**
        - It is key that the medical practitioner sees the information that the LLM can see, because LLMs do have a habit of making things up... 
    - **Add a Front-end User Interface**
    - **Create more detailed medical history using a range of resource types (e.g. Conditions, Immunizations, Observations, Medications and more)**
    - **Improve Patient ID collection** - At the moment the method to give the Patient ID is very clunky and involves knowing the patient by their ID, not by any easily identifiable systems. This could be improved by adding a beginning form or interaction that asks for full name, address or DOB to identify the patient without needing to know the ID>
    - **Using an Agentic method to add functionality to the chatbot** - You could look at LangChain or LangGraph documentation to have a better idea of this. 


On the final point, here I have used DocumentReference resources because this is one of the few plain text resources with clinically relevant notes. Even in this example, I had to generate the clinical notes myself using an LLM. A more complete example of this may involve creating plain-text strings of medical notes using other resources available. For example, you could group FHIR resources by year, take clinically relevant resources [ Condition, Observation, AllergyIntolerance, Procedures, Immunization, CarePlan], create strings representing the clinical information per year and use this for the vector search method we have implemented above. 

A nice example of a complete project with a similar set-up is available on the [Open Exchange](https://openexchange.intersystems.com/package/FHIR-Data-Explorer-with-Hybrid-Search-and-AI-Summaries-1) by Pietro Di Leo. There are some key differences from this demo, for example the tabular data is created within Python and directly loaded into a SQL database. 


